In [1]:
import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy
import pymysql
pymysql.install_as_MySQLdb()

## National Footprint Accounts 2018 from kaggle.com

In [26]:
NFA_df=pd.read_csv("NFA_2018.csv")
print(NFA_df.shape)
print(NFA_df.columns)
NFA_df.head()

(87020, 15)
Index(['country', 'ISO alpha-3 code', 'UN_region', 'UN_subregion', 'year',
       'record', 'crop_land', 'grazing_land', 'forest_land', 'fishing_ground',
       'built_up_land', 'carbon', 'total', 'Percapita GDP (2010 USD)',
       'population'],
      dtype='object')


,country,ISO alpha-3 code,UN_region,UN_subregion,year,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,Percapita GDP (2010 USD),population
0,Armenia,ARM,Asia,Western Asia,1992,BiocapPerCap,1.611286e-01,0.135023,0.083836,0.013718,0.033669,0.000000e+00,4.273741e-01,949.033,3449000
1,Armenia,ARM,Asia,Western Asia,1992,BiocapTotGHA,5.558130e+05,465763.337400,289190.662300,47320.224590,116139.598200,0.000000e+00,1.474227e+06,949.033,3449000
2,Armenia,ARM,Asia,Western Asia,1992,EFConsPerCap,3.909225e-01,0.189137,0.000001,0.004138,0.033669,1.112225e+00,1.730092e+00,949.033,3449000
3,Armenia,ARM,Asia,Western Asia,1992,EFConsTotGHA,1.348487e+06,652429.066600,4.327841,14272.803690,116139.598200,3.836620e+06,5.967954e+06,949.033,3449000
4,Armenia,ARM,Asia,Western Asia,1992,EFExportsPerCap,1.124910e-03,0.002283,0.000000,0.000438,0.000000,4.819043e-02,5.203676e-02,949.033,3449000


In [25]:
countries_df=NFA_df[["ISO alpha-3 code","country"]].drop_duplicates(keep="first")

# find missing values in "code" column
print(len(countries_df["ISO alpha-3 code"].unique()) == len(countries_df["country"].unique()))
drop_list=countries_df[countries_df["ISO alpha-3 code"].isna()]["country"]
print(drop_list)
countries_df=countries_df[~(countries_df["country"].isin(drop_list))]

False
23020      Ethiopia PDR
69780    Sudan (former)
85700             World
Name: country, dtype: object


In [22]:
NFA_2_df=NFA_df[~(NFA_df["country"].isin(drop_list))]

In [21]:
regions_df=NFA_2_df[['country','UN_region','UN_subregion']].drop_duplicates(keep="first").reset_index(drop=True)

In [23]:
population_df=NFA_2_df[["country","year","population"]].drop_duplicates(keep="first").reset_index(drop=True)

In [24]:
records_list_df=NFA_df["record"].drop_duplicates(keep="first")

In [27]:
e_footprint_df=NFA_2_df[["country","year",'record', 'crop_land', 'grazing_land', 'forest_land', 'fishing_ground',
       'built_up_land', 'carbon']].loc[NFA_df["record"].isin(["BiocapTotGHA"," EFConsTotGHA","EFExportsTotGHA","EFImportsTotGHA","EFProdTotGHA"])]

## countries' land area from wikipedia page

In [11]:
url="https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_area"
country_area_list=pd.read_html(url)
country_area_df=country_area_list[0]
columns=list(country_area_df.loc[0])
print(columns)
columns=['Rank', 'country', 'Total in km2 (mi2)', 'Land in km2 (mi2)', 'Water in km2 (mi2)', '% water', 'Notes']
country_area_df.columns=columns
country_area_df=country_area_df.iloc[2:]
country_area_df=country_area_df.drop(columns="Rank").reset_index(drop=True)
print(country_area_df.shape)
country_area_df.head()

['Rank', 'Sovereign state/dependency', 'Total in km2 (mi2)', 'Land in km2 (mi2)', 'Water in km2 (mi2)', '% water', 'Notes']
(261, 6)


,country,Total in km2 (mi2),Land in km2 (mi2),Water in km2 (mi2),% water,Notes
0,Russia,"(6,601,670)","(6,323,482)","(278,200)",4.21,Largest country in the world (10.995% of the w...
1,Antarctica,"(5,400,000)","(5,400,000)",(0),0,"13,720,000 km2 (5,300,000 sq mi) (98%) of land..."
2,Canada,"(3,855,100)","(3,511,023)","(344,080)",8.93,Largest English and French-speaking country an...
3,China,"(3,705,407)","(3,600,950)","(104,460)",2.82,Second largest country in Asia (though the lar...
4,United States,"(3,677,649)[7] –(3,796,742)","(3,531,905) –(3,531,925)[7]","(145,724)[7] –(264,837)",3.96–6.97,Second largest English-speaking country and th...


In [12]:
country_area_df=country_area_df[country_area_df["country"].isin(countries_df["country"])]
country_area_df.shape

(158, 6)

In [19]:
area_df=pd.merge(countries_df, country_area_df,on="country",how="left").sort_values("country")

area_df.shape
area_df[area_df["country"].isin(["Denmark"])]

,ISO alpha-3 code,country,Total in km2 (mi2),Land in km2 (mi2),Water in km2 (mi2),% water,Notes
43,DNK,Denmark,"(16,639)","(16,384)",(250),1.53,"Metropolitan Denmark only, without Greenland a..."
42,DNK,Denmark,"(857,183)","(857,175)",(8.1),0,UN figure does not includes entire Kingdom of ...


In [ ]:
trial2=countries_df[(~countries_df["country"].isin(country_area_df["country"]))]["country"]
trial2

In [ ]:
rds_connection_string = "localhost:mohammad@127.0.0.1/co2_emissions"
engine = create_engine(f'mysql://{rds_connection_string}')

## World CO2 emissions from consumption of energy from theguardian.com

In [28]:
co2 = pd.read_excel('World CO2 emissions from consumption of energy.xlsx',sheet_name= 'Total Carbon Dioxide Emissions')

In [29]:
co2.columns = co2.iloc[0,]

In [30]:
co2.reset_index(inplace= True)

In [31]:
co2 = co2.reindex(co2.index.drop(0))

In [32]:
co2.rename(columns = {'level_0':'rank_2009','level_1':'rank_2008','Rank, 2006':'rank_2006',
                     'ISO country code': 'iso_code'},inplace = True)
co2.columns.name = None

In [33]:
clean = co2.loc[~co2['iso_code'].isnull(),]

In [41]:
# Table!! Create the ISO Code and Country List
iso_country = clean[['iso_code','Country']].drop_duplicates().reset_index(drop = True).reset_index().rename(columns = {'Country':'country','index':'id'})
iso_country.head()

# remove duplicated isocode and year combinations
iso_country = iso_country.loc[~iso_country.duplicated(subset=['iso_code','country'])]

# UM map to both 'Wake Island' and U.S. Pacific Islands. After research, update UM for U.S. Pacific Islands to UM1
iso_country[iso_country['country'] == 'U.S. Pacific Islands']['iso_code'] = 'UM1'
iso_country.head()


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,id,iso_code,country
0,0,CN,China
1,1,US,United States
2,2,IN,India
3,3,RU,Russia
4,4,JP,Japan


In [85]:
iso_country[iso_country['country'] == 'cape Verde']['country'] = 'Cabo Verde'
iso_country[iso_country['country'] == 'Congo, Dem Rep']['country'] = ' Congo, Democratic Republic of'
iso_country[iso_country['country'] == 'Iran']['country'] = 'Iran, Islamic Republic of'
iso_country[iso_country['country'] == 'Korea, North']['country'] = "Korea, Democratic People's Republic of"
iso_country[iso_country['country'] == 'Korea, South']['country'] = "Korea, Republic of"
iso_country[iso_country['country'] == 'Laos']['country'] = "Lao People's Democratic Republic"
iso_country[iso_country['country'] == 'Libya']['country'] = "Libyan Arab Jamahiriya"
#iso_country[iso_country['country'] == 'Korea, North']['country'] = "Korea, Democratic People's Republic of"
trial3=pd.merge(iso_country,countries_df,how="outer",on="country").sort_values("country").reset_index(drop=True)
trial4=trial3[trial3.isnull().any(axis=1)]
print(trial4)
print(iso_country[iso_country['country'] == 'cape Verde']['country'] )

        id iso_code                                 country ISO alpha-3 code
3    180.0       AS                          American Samoa              NaN
5    191.0       AQ                              Antarctica              NaN
27   105.0       BN                                  Brunei              NaN
28     NaN      NaN                       Brunei Darussalam              BRN
31    92.0       MM                                   Burma              NaN
33     NaN      NaN                              Cabo Verde              CPV
37   188.0       CV                              Cape Verde              NaN
46   143.0       CD                          Congo, Dem Rep              NaN
47     NaN      NaN           Congo, Democratic Republic of              COD
50   114.0       CI               Cote dIvoire (IvoryCoast)              NaN
55     NaN      NaN                          Czechoslovakia             CSHH
56     NaN      NaN                           Côte d'Ivoire              CIV

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [48]:
##### rank_list = ['rank_2009','rank_2008','rank_2006','% change,  2008 to 2009','% change 2000 to 2009','Change in place, 2008 to 2009']
clean_rank_change = clean[[*rank_list,'iso_code']].set_index('iso_code')
clean_rank_change.head()

,rank_2009,rank_2008,rank_2006,"% change, 2008 to 2009",% change 2000 to 2009,"Change in place, 2008 to 2009"
iso_code,,,,,,
CN,1,1,2,13.3,170.6,SAME
US,2,2,1,-7,-7.5,SAME
IN,3,4,4,8.7,59.7,UP
RU,4,3,3,-7.4,1,DOWN
JP,5,5,5,-9.7,-8.6,SAME


In [38]:
#remove the percentage changes
clean_rank_new = clean_rank_change[['rank_2009','rank_2008','rank_2006']].rename(columns = lambda x : x.replace('rank_',''))

clean_rank_new = clean_rank_new.stack().reset_index().rename(columns = {'level_1':'year',0:'rank'})


clean_rank_new = clean_rank_new[~clean_rank_new.duplicated(subset = ['iso_code','year'])]
clean_rank_new.head()

,iso_code,year,rank
0,CN,2009,1
1,CN,2008,1
2,CN,2006,2
3,US,2009,2
4,US,2008,2


In [37]:
##Table!! Create the Country, Year, and CO2 Emission table
clean_co2 = clean.drop(rank_list,axis = 1)
clean_co2 = clean_co2.set_index('iso_code').drop('Country',axis = 1)
clean_co2 = clean_co2.stack().reset_index().rename(columns = {'level_1':'year',0:'co2_emission'})

# remove duplicated isocode and year combinations
clean_co2 = clean_co2.loc[~clean_co2.duplicated(subset=['iso_code','year'])]